In [63]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

CUDA available: True
GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [65]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [67]:
dataset =pd.read_csv("Churn_Modelling.csv")

In [69]:
X = dataset.iloc[:,3:13]
y = dataset.iloc[:,13]

In [71]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [73]:
#feature engeniering
geography = pd.get_dummies(X['Geography'],drop_first=True)
gender = pd.get_dummies(X['Gender'],drop_first=True)

In [75]:
X=X.drop(['Geography','Gender'],axis=1)

In [77]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,42,2,0.00,1,1,1,101348.88
1,608,41,1,83807.86,1,0,1,112542.58
2,502,42,8,159660.80,3,1,0,113931.57
3,699,39,1,0.00,2,0,0,93826.63
4,850,43,2,125510.82,1,1,1,79084.10


In [79]:
X=pd.concat([X,geography,gender],axis=1).astype(int)

In [81]:
from sklearn.model_selection import train_test_split

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [85]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc =StandardScaler()

In [87]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [89]:
X_train

array([[ 0.16958176, -0.46460796,  0.00666099, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-2.30455945,  0.30102557, -1.37744033, ...,  1.75486502,
        -0.57369368,  0.91601335],
       [-1.19119591, -0.94312892, -1.031415  , ..., -0.5698444 ,
        -0.57369368, -1.09168714],
       ...,
       [ 0.9015152 , -0.36890377,  0.00666099, ..., -0.5698444 ,
        -0.57369368,  0.91601335],
       [-0.62420521, -0.08179119,  1.39076231, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-0.28401079,  0.87525072, -1.37744033, ...,  1.75486502,
        -0.57369368, -1.09168714]])

In [91]:
X_train.shape

(8000, 11)

In [93]:
#its time for the ann
import torch.nn as nn

In [123]:
model = nn.Sequential(
    nn.Linear(11, 11),   # "input layer" in Keras terms
    nn.ReLU(),

    nn.Linear(11, 7),    # hidden layer 1
    nn.ReLU(),

    nn.Linear(7, 6),     # hidden layer 2
    nn.ReLU(),

    nn.Linear(6, 1),     # output layer
    nn.Sigmoid()
)

In [125]:
device = 'cuda' if torch.cuda.is_available else "cpu"
model = model.to(device)

In [127]:
criterion = nn.BCELoss()

In [129]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [131]:
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)



C:\Users\Piyush Singh\AppData\Local\Temp\ipykernel_18132\402826599.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
C:\Users\Piyush Singh\AppData\Local\Temp\ipykernel_18132\402826599.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)


In [133]:
print(type(X_train))
print(X_train.shape)
print(type(y_train))
print(y_train.shape)


<class 'torch.Tensor'>
torch.Size([8000, 11])
<class 'torch.Tensor'>
torch.Size([8000, 1])


In [147]:
epochs =500

for epoch in range(epochs):
    model.train()
    
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():4f}")

Epoch 10, Loss: 0.474660
Epoch 20, Loss: 0.465381
Epoch 30, Loss: 0.457682
Epoch 40, Loss: 0.451034
Epoch 50, Loss: 0.445309
Epoch 60, Loss: 0.440153
Epoch 70, Loss: 0.435503
Epoch 80, Loss: 0.431232
Epoch 90, Loss: 0.427331
Epoch 100, Loss: 0.423840
Epoch 110, Loss: 0.420698
Epoch 120, Loss: 0.417827
Epoch 130, Loss: 0.415102
Epoch 140, Loss: 0.412434
Epoch 150, Loss: 0.409763
Epoch 160, Loss: 0.407009
Epoch 170, Loss: 0.404251
Epoch 180, Loss: 0.401520
Epoch 190, Loss: 0.398827
Epoch 200, Loss: 0.396201
Epoch 210, Loss: 0.393703
Epoch 220, Loss: 0.391364
Epoch 230, Loss: 0.389053
Epoch 240, Loss: 0.386864
Epoch 250, Loss: 0.384782
Epoch 260, Loss: 0.382885
Epoch 270, Loss: 0.381115
Epoch 280, Loss: 0.379442
Epoch 290, Loss: 0.377773
Epoch 300, Loss: 0.376070
Epoch 310, Loss: 0.374310
Epoch 320, Loss: 0.372494
Epoch 330, Loss: 0.370577
Epoch 340, Loss: 0.368559
Epoch 350, Loss: 0.366425
Epoch 360, Loss: 0.364282
Epoch 370, Loss: 0.362150
Epoch 380, Loss: 0.360055
Epoch 390, Loss: 0.35

In [149]:
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
# y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)i still think it can be improved but leave it whatver


C:\Users\Piyush Singh\AppData\Local\Temp\ipykernel_18132\1219010773.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype=torch.float32).to(device)


TypeError: must be real number, not builtin_function_or_method

In [151]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = (y_pred > 0.5).float()

In [153]:
accuracy = (y_pred == y_test).float().mean()
print("Test Accuracy:", accuracy.item())

Test Accuracy: 0.8540000319480896
